In [1]:
import pandas as pd
import numpy as np
from lifelines import CoxPHFitter
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score


In [2]:
## CALCULATE STROMA SCORE
# df = pd.read_excel("../data/TCGA_MEASUREMENTS.xlsx")

def specificity(y_true, y_pred):
    TN = np.sum(np.logical_and(y_pred == 0, y_true == 0))

    N = len(y_true) -np.sum(y_true)
    
    return TN/N

def calculate_stroma_score(df):
#     df = pd.read_excel(data_dir)
    no_rows = len(df)

    x = df.to_numpy()
    x[:, -1] = x[:, -1] / 365.35  # convert 'days to event' to 'years to event'
    x[:, 11] = x[:, 11] / 10  # convert 'years to birth' to 'decades to birth'
    x = np.append(x, np.zeros((no_rows, 1)), axis=1)  # add column for HD score

    # Calculate allWeights
    classes_col = ["ADI", "BACK", "DEB", "LYM", "MUC", "MUS", "NORM", "STR", "TUM"]
    df["years_to_event"] = (
        df["days_to_event"] / 365.25
    )  # convert 'days to event' to 'years to event'
    df["decades_to_birth"] = (
        df["years_to_birth"] / 10
    )  # convert 'years to birth' to 'decades to birth'
    y = df[["years_to_event", "vital_status"]]

    cph_models = [
        CoxPHFitter().fit(
            pd.concat([df[col], y], axis=1), "years_to_event", "vital_status"
        )
        for col in classes_col
    ]
    allWeights = np.array([float(cph.summary["exp(coef)"]) for cph in cph_models])

    # Calculate allCuts?
    # allCuts = np.array(
    #     [
    #         0.00056,
    #         0.00227,
    #         0.03151,
    #         0.00121,
    #         0.01123,
    #         0.02359,
    #         0.06405,
    #         0.00122,
    #         0.99961,
    #     ]
    # )  # Youden cuts

    allCuts = []

    for i, col_name in enumerate(classes_col):
        class_score = df[col_name]
        max_index = 0
        median = np.median(class_score)
        for j, score in enumerate(class_score):
            if j == 0:
                allCuts.append(score)

            preds = np.greater_equal(class_score, score).astype(int)
            sens = recall_score(df["vital_status"], preds)
            spec = specificity(df["vital_status"], preds)
            if (sens + spec) > max_index:
                max_index = sens + spec
                allCuts[i] = score
            elif sens + spec == max_index:
                if abs(score - median) < abs(allCuts[i] - median):
                    allCuts[i] = score
                    max_index = score

    # Calculate stroma score
    scoreIndices = (np.argwhere(allWeights >= 1)).flatten()
    for i in scoreIndices:
        x[:, -1] = (
            x[:, -1] + (x[:, i + 1] >= allCuts[i]) * allWeights[i]
        )  # +1 retrieve column number in x
    medianTrainingSet = np.median(x[:, -1])
    x[:, -1] = (x[:, -1] >= medianTrainingSet) * 1
    stroma_score = x[:, -1]

    runs = ["All stages", "Stage i", "Stage ii", "Stage iii", "Stage iv"]
    df["stroma_score"] = stroma_score

    selected_columns = ["stroma_score", "cleanstage", "gender", "decades_to_birth"]
    df_mv = df.dropna(subset=["cleanstage", "decades_to_birth"])
    y_mv = df_mv[["years_to_event", "vital_status"]]

    mv_cox = CoxPHFitter().fit(
        pd.concat([df_mv[selected_columns], y_mv], axis=1),
        "years_to_event",
        "vital_status",
        formula="stroma_score + cleanstage + C(gender) +decades_to_birth",
    )
    # mv_cox.print_summary()
    print(mv_cox.hazard_ratios_)

    stroma_HR = mv_cox.hazard_ratios_[3]
    lci = np.exp(mv_cox.confidence_intervals_["95% lower-bound"][3])
    hci = np.exp(mv_cox.confidence_intervals_["95% upper-bound"][3])
    p = mv_cox._compute_p_values()[3]
    print(
        f"___Deep Stroma Score Hazard Ratio, {runs[0]}___ \n Stroma HR: {stroma_HR}; \n CI: [{lci}, {hci}]; \n p: {p}"
    )

    return stroma_score


## Original excel

In [3]:
df = pd.read_excel("../data/TCGA_MEASUREMENTS.xlsx")
df["years_to_event"] = df["days_to_event"]/365.25
df["decades_to_birth"] = df["years_to_birth"]/10


In [4]:
classes_col = ["ADI","BACK", "DEB", "LYM", "MUC","MUS", "NORM", "STR","TUM"]

In [5]:
y = df[["years_to_event","vital_status"]]
y_days = df[["days_to_event","vital_status"]]

In [6]:
df.head()

,ID,ADI,BACK,DEB,LYM,MUC,MUS,NORM,STR,TUM,...,histological_type,hypermutated,methylation_subtype,CAF_SCORE,percent_stromal_cells,RF_predictedCMS,cleanstage,days_to_event,years_to_event,decades_to_birth
0,TCGA-CM-6675,0.000284,0.000204,0.090979,0.000544,0.013228,0.007923,0.101498,0.086859,0.798861,...,colon adenocarcinoma,0.0,CIMP-H,2.080628,12,CMS1,4.0,397,1.086927,3.5
1,TCGA-AY-A8YK,0.000324,0.000290,0.004827,0.013253,0.004651,0.002694,0.066702,0.140175,0.767085,...,colon adenocarcinoma,NaN,NaN,1.635184,7,NaN,4.0,573,1.568789,4.4
2,TCGA-CM-4747,0.001219,0.004085,0.197126,0.337597,0.002646,0.003099,0.158479,0.447476,0.485696,...,colon adenocarcinoma,0.0,CIMP-L,2.024608,15,NaN,4.0,761,2.083504,4.7
3,TCGA-DY-A1DG,0.003772,0.001362,0.188463,0.002173,0.012698,0.063342,0.096374,0.012094,0.619722,...,rectal adenocarcinoma,NaN,Cluster3,0.990050,0,NaN,4.0,1566,4.287474,7.5
4,TCGA-CM-5862,0.007687,0.006287,0.386051,0.086920,0.132158,0.130372,0.076991,0.102107,0.481279,...,colon adenocarcinoma,0.0,Cluster3,1.944954,0,CMS2,4.0,153,0.418891,8.0


In [7]:
selected_columns = ["obtained_scores" ,"cleanstage", "gender", "decades_to_birth" ]
df_mv = df.copy()
df_mv = df_mv.dropna(subset=["cleanstage", "decades_to_birth"])
y_mv = df_mv[["years_to_event","vital_status"]]

## Our values

In [8]:
df_avg = pd.read_csv("../data/TCGA_SA_data_average.csv",)

df_avg["years_to_event"] = df_avg["days_to_event"]/365.25 # convert 'days to event' to 'years to event'
df_avg["decades_to_birth"] = df_avg["years_to_birth"]/10 # 
df_avg = df_avg[df_mv.columns]
df_avg = df_avg.dropna(subset=["cleanstage", "decades_to_birth"])

In [9]:
df_org_same_patients = df_mv.loc[df_mv['ID'].isin(df_avg["ID"])]
y_org = df_org_same_patients[["years_to_event","vital_status"]]
y_avg = df_avg[["years_to_event","vital_status"]]

In [10]:
df_avg["obtained_scores"] = calculate_stroma_score(df_avg)

covariate
C(gender)[T.male]    0.766476
cleanstage           2.476036
decades_to_birth     1.394927
stroma_score[T.1]    2.142212
Name: exp(coef), dtype: float64
___Deep Stroma Score Hazard Ratio, All stages___ 
 Stroma HR: 2.1422118704839512; 
 CI: [1.3083162765591028, 3.5076164535013636]; 
 p: 0.002460517929049559


In [11]:
df_org_same_patients["obtained_scores"] = calculate_stroma_score(df_org_same_patients)

/tmp/ipykernel_6761/2675198603.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["years_to_event"] = (
/tmp/ipykernel_6761/2675198603.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["decades_to_birth"] = (


covariate
C(gender)[T.male]    0.796488
cleanstage           2.425369
decades_to_birth     1.434641
stroma_score[T.1]    2.156620
Name: exp(coef), dtype: float64
___Deep Stroma Score Hazard Ratio, All stages___ 
 Stroma HR: 2.15662012399965; 
 CI: [1.233910464046312, 3.769325647817588]; 
 p: 0.0069803856574124174


/tmp/ipykernel_6761/2675198603.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["stroma_score"] = stroma_score
/tmp/ipykernel_6761/2885598111.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_org_same_patients["obtained_scores"] = calculate_stroma_score(df_org_same_patients)


In [12]:
# print(mv_regression_df)
mv_cox_orginal_data = CoxPHFitter().fit(pd.concat([df_org_same_patients[selected_columns], y_org], axis=1), "years_to_event", "vital_status", 
                           formula = "obtained_scores + cleanstage + C(gender) +decades_to_birth" )


In [13]:
mv_cox_orginal_data.print_summary()

<lifelines.CoxPHFitter: fitted with 338 total observations, 264 right-censored observations>
             duration col = 'years_to_event'
                event col = 'vital_status'
      baseline estimation = breslow
   number of observations = 338
number of events observed = 74
   partial log-likelihood = -344.34
         time fit was run = 2022-12-07 18:21:02 UTC

---
                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                    
C(gender)[T.male]     -0.23       0.80       0.24            -0.69             0.23                 0.50                 1.26
cleanstage             0.89       2.43       0.14             0.61             1.16                 1.84                 3.20
decades_to_birth       0.36       1.43       0.10             0.17             0.55                 1.18                 1.74
obtained_scores[T.1]   0.77       2.16       0.28             0.21             1.33                 1.23                 3.77

                       cmp to     z      p   -log2(p)
covariate                                            
C(gender)[T.male]        0.00 -0.96   0.33       1.58
cleanstage               0.00  6.30 <0.005      31.61
decades_to_birth         0.00  3.65 <0.005      11.89
obtained_scores[T.1]     0.00  2.70   0.01       7.16
---
Concordance = 0.76
Partial AIC = 696.68
log-likelihood ratio test = 60.14 on 4 df
-log2(p) of ll-ratio test = 38.43

In [16]:
mv_cox_avg = CoxPHFitter().fit(pd.concat([df_avg[selected_columns], y_avg], axis=1), "years_to_event", "vital_status", 
                           formula = "obtained_scores + cleanstage + C(gender) +decades_to_birth" )


In [17]:
mv_cox_avg.print_summary()

<lifelines.CoxPHFitter: fitted with 338 total observations, 264 right-censored observations>
             duration col = 'years_to_event'
                event col = 'vital_status'
      baseline estimation = breslow
   number of observations = 338
number of events observed = 74
   partial log-likelihood = -343.63
         time fit was run = 2022-12-07 18:21:41 UTC

---
                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                    
C(gender)[T.male]     -0.27       0.77       0.24            -0.73             0.20                 0.48                 1.22
cleanstage             0.91       2.48       0.14             0.64             1.17                 1.89                 3.24
decades_to_birth       0.33       1.39       0.10             0.13             0.53                 1.14                 1.70
obtained_scores[T.1]   0.76       2.14       0.25             0.27             1.25                 1.31                 3.51

                       cmp to     z      p   -log2(p)
covariate                                            
C(gender)[T.male]        0.00 -1.12   0.26       1.94
cleanstage               0.00  6.64 <0.005      34.89
decades_to_birth         0.00  3.30 <0.005       9.99
obtained_scores[T.1]     0.00  3.03 <0.005       8.67
---
Concordance = 0.76
Partial AIC = 695.26
log-likelihood ratio test = 61.56 on 4 df
-log2(p) of ll-ratio test = 39.42

## Our data - highest probability

In [27]:
df_highest = pd.read_csv("../data/TCGA_SA_data_highest_tum.csv",)

df_highest["years_to_event"] = df_highest["days_to_event"]/365.25 # convert 'days to event' to 'years to event'
df_highest["decades_to_birth"] = df_highest["years_to_birth"]/10 # 
df_highest = df_highest[df_mv.columns]
df_highest = df_highest.dropna(subset=["cleanstage", "decades_to_birth"])
df_highest["obtained_scores"] = calculate_stroma_score(df_highest)

covariate
C(gender)[T.male]    0.845069
cleanstage           2.479205
decades_to_birth     1.420707
stroma_score[T.1]    1.060802
Name: exp(coef), dtype: float64
___Deep Stroma Score Hazard Ratio, All stages___ 
 Stroma HR: 1.0608015081227917; 
 CI: [0.6450475591130356, 1.7445222817103883]; 
 p: 0.8161061470125706


In [28]:
y_highest  = df_highest[["years_to_event","vital_status"]]

In [29]:
mv_cox_highest = CoxPHFitter().fit(pd.concat([df_highest[selected_columns], y_highest], axis=1), "years_to_event", "vital_status", 
                           formula = "obtained_scores + cleanstage + C(gender) +decades_to_birth" )


In [30]:
mv_cox_highest.print_summary()

<lifelines.CoxPHFitter: fitted with 338 total observations, 264 right-censored observations>
             duration col = 'years_to_event'
                event col = 'vital_status'
      baseline estimation = breslow
   number of observations = 338
number of events observed = 74
   partial log-likelihood = -348.44
         time fit was run = 2022-12-07 18:24:44 UTC

---
                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                    
C(gender)[T.male]     -0.17       0.85       0.24            -0.63             0.29                 0.53                 1.34
cleanstage             0.91       2.48       0.14             0.64             1.18                 1.89                 3.25
decades_to_birth       0.35       1.42       0.10             0.15             0.55                 1.17                 1.73
obtained_scores[T.1]   0.06       1.06       0.25            -0.44             0.56                 0.65                 1.74

                       cmp to     z      p   -log2(p)
covariate                                            
C(gender)[T.male]        0.00 -0.71   0.48       1.07
cleanstage               0.00  6.56 <0.005      34.12
decades_to_birth         0.00  3.50 <0.005      11.07
obtained_scores[T.1]     0.00  0.23   0.82       0.29
---
Concordance = 0.75
Partial AIC = 704.88
log-likelihood ratio test = 51.95 on 4 df
-log2(p) of ll-ratio test = 32.72

## Only raw probabilities from model

In [33]:
cols_with_classes = ["cleanstage", "gender", "decades_to_birth", "ADI", "BACK", "DEB", "LYM", "MUC", "MUS", "NORM", "STR", "TUM"]
mv_cox_raw = CoxPHFitter().fit(pd.concat([df_org_same_patients[cols_with_classes], y_org], axis=1), "years_to_event", "vital_status", 
                           formula = "ADI + BACK + DEB + LYM + MUC+ MUS + NORM + STR + TUM + cleanstage + C(gender) +decades_to_birth" )

In [35]:
mv_cox_raw.print_summary()

<lifelines.CoxPHFitter: fitted with 338 total observations, 264 right-censored observations>
             duration col = 'years_to_event'
                event col = 'vital_status'
      baseline estimation = breslow
   number of observations = 338
number of events observed = 74
   partial log-likelihood = -342.87
         time fit was run = 2022-12-07 18:34:23 UTC

---
                    coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                 
ADI                 1.02       2.76       2.51            -3.91             5.94                 0.02               381.49
BACK               -1.50       0.22       7.88           -16.94            13.93                 0.00             1.12e+06
C(gender)[T.male]  -0.27       0.76       0.25            -0.76             0.22                 0.47                 1.24
DEB                 2.19       8.94       1.00             0.23             4.15                 1.26                63.43
LYM                -0.00       1.00       0.77            -1.51             1.51                 0.22                 4.52
MUC                -0.19       0.83       1.30            -2.73             2.35                 0.07                10.51
MUS                 0.20       1.23       0.92            -1.60             2.01                 0.20                 7.44
NORM               -0.08       0.93       0.90            -1.84             1.69                 0.16                 5.40
STR                -0.52       0.59       1.08            -2.63             1.59                 0.07                 4.89
TUM                -0.70       0.50       0.65            -1.98             0.58                 0.14                 1.79
cleanstage          0.97       2.65       0.15             0.69             1.26                 1.99                 3.53
decades_to_birth    0.35       1.43       0.10             0.16             0.55                 1.17                 1.74

                    cmp to     z      p   -log2(p)
covariate                                         
ADI                   0.00  0.40   0.69       0.54
BACK                  0.00 -0.19   0.85       0.24
C(gender)[T.male]     0.00 -1.09   0.28       1.86
DEB                   0.00  2.19   0.03       5.13
LYM                   0.00 -0.00   1.00       0.00
MUC                   0.00 -0.15   0.88       0.18
MUS                   0.00  0.22   0.82       0.28
NORM                  0.00 -0.09   0.93       0.10
STR                   0.00 -0.49   0.63       0.67
TUM                   0.00 -1.07   0.28       1.81
cleanstage            0.00  6.67 <0.005      35.23
decades_to_birth      0.00  3.49 <0.005      11.00
---
Concordance = 0.76
Partial AIC = 709.74
log-likelihood ratio test = 63.09 on 12 df
-log2(p) of ll-ratio test = 27.28